<a href="https://colab.research.google.com/github/CharlotteY2003/Simply-Recipes-Web-Scraper/blob/main/Advanced_Simply_Recipes_Web_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Updates


*   Larger dataset of recipes
*   Instead of manually inputting recipe cuisine url, only need url of main Simply Recipes site
*   Avoid repeat recipes by checking if names are identical after input to datatable
*   Avoid article "how tos" by checking if ingredient section exists
* Takes around 2 hours to scrape recipe dataset of 1000 recipes

# Initialization

In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup #https://www.crummy.com/software/BeautifulSoup/bs4/doc/#sibling-generators
import pandas as pd
import numpy as np

from time import sleep
from random import randint

In [2]:
titles = []
prep = []
cats = []
servings = []
ingreds = []

In [3]:
url = 'https://www.simplyrecipes.com/recipes-5090746'

# Retrieve Information from Recipe Site

In [4]:
#gets all recipes from recipe category page
def getRecipe(item):

  url = item.parent['href']
  print(url)

  #get ingredients
  ingred_list = getFood(url)
  if (ingred_list is None):
    return
  else:
    ingreds.append(ingred_list)
  

  #get recipe name
  name = getName(item)
  if name in titles:
    del ingreds[-1]
    return
  else:
    titles.append(name)

  #vegetarian or not
  cats.append(Veg(item))
  
  #get prep time
  prep.append(getPrep(item))

  
  #get recipe's serving number
  serving = getServingNum(url)
  servings.append(serving)

In [5]:
def getName(thing):
  name = thing.contents[1].contents[3].contents[1].text
  name = name.strip()
  return name

In [6]:
def Veg(thing):
  text = str(thing.div.contents[1])
  tag = BeautifulSoup(text, 'html.parser').div
  cat = tag['data-tax-tag']
  return cat

In [7]:
def getPrep(thing):
  try:
    time = thing.contents[3].contents[1].span.text
  except:
    time = "0"
  time = time.strip()
  return time

In [8]:
def getServingNum(link):
  results = requests.get(link)
  soup = BeautifulSoup(results.text, "html.parser")
  recipe_div = soup.find_all('div', class_="loc recipe-serving project-meta__recipe-serving")
  try:
    new_item = recipe_div[0]
  except:
    recipe_div = soup.find_all('div', class_ = 'loc recipe-yield project-meta__recipe-yield')
  #print(recipe_div)

  if not recipe_div:
    return None
  else:
    new_item = recipe_div[0]
    #get recipe's serving number
    serving = new_item.contents[1].contents[3].text#[5]#.div.contents[3].div.contents[1].contents[3].text 
    serving = serving.replace('\n', '')
    #print(serving)
    return serving
  

In [9]:
#gets each ingredient from individual recipe
def getFood(link):
  temp_arr = []
  results = requests.get(link)
  soup = BeautifulSoup(results.text, "html.parser")
  recipe_div = soup.find('ul', class_='comp ingredient-list simple-list simple-list--circle ')
  try:
    food = recipe_div.contents[1]
  except:
    if soup.find('ul', class_='structured-ingredients__list text-passage'):
      recipe_div = soup.find('ul', class_='structured-ingredients__list text-passage')
    else:
      return None
  
  food = recipe_div.contents[1]
  temp_arr.append(food.text.strip())

  score = 0
  for sibling in food.next_siblings:
    if (not score%2 == 0):
      #print(sibling.text)
      food = sibling.text.strip()
      temp_arr.append(food)
    score = score +1

  return temp_arr

In [10]:
#loops through all recipes on that page, extracting recipe name, vegatarian or not, prep time, 
#number of servings, and ingredients
def getAllRecipes(recipe_div):
  for container in recipe_div:
    getRecipe(container) 
    sleep(randint(2,5))

In [11]:
#creates table with given categories
def createTable():
  recipes = pd.DataFrame({
    'titles': titles,
    'category': cats,
    'Prep time': prep,
    'servings': servings,
    'Ingreds': ingreds  
}) 
  recipes.to_csv('recipes.csv')
  return recipes

In [12]:
# Gets all specified information from multiple pages (urls) of website and creates a table out of them
#New goal: using only url of simply website homepage, get all specified info from multiple pages and create table out of them

#Does not account for sections embedded into sections - maybe use recursive functions?
# Defunct function
def scrapeRecipes(url):
  results = requests.get(url)
  soup = BeautifulSoup(results.text, "html.parser")
  all_sections = soup.find_all('div', class_="loc section-header")

  for i in range(len(all_sections)):
    print(i)
    print(all_sections[i])
    url = all_sections[i].a['href']

    #get all recipes in that section
    results = requests.get(url)
    soup = BeautifulSoup(results.text, "html.parser")
    recipe_div = soup.find_all('div', class_="card__wrapper")
    getAllRecipes(recipe_div)

  recipe = createTable()
  return recipe

In [13]:
#check
def getEarlySec(link):

  if (len(titles) > 1500):
    return
  else:
    print(len(titles))
    results = requests.get(link)
    soup = BeautifulSoup(results.text, "html.parser")
    all_sections = soup.find_all('div', class_="loc section-header")
    
    if not all_sections:
      recipe_div = soup.find_all('div', class_="card__wrapper")
      getAllRecipes(recipe_div)
    
    else:
      for i in range(len(all_sections)):
        new_url = all_sections[i].a['href'] 
        getEarlySec(new_url)

In [14]:
def recipeScrape(url):
  getEarlySec(url)
  print("recursive finished")
  recipe = createTable()
  print('table creation finished')
  return recipe

# Function being Called

In [15]:
table = recipeScrape(url)

0
0
https://www.simplyrecipes.com/how-to-store-onions-5191927
https://www.simplyrecipes.com/how-to-store-garlic-5191924
https://www.simplyrecipes.com/how-to-store-fresh-ginger-5191919
https://www.simplyrecipes.com/french-buttercream-recipe-5191534
https://www.simplyrecipes.com/hetty-mckinnon-s-asian-pantry-essentials-5190920
https://www.simplyrecipes.com/everything-you-want-to-know-about-impossible-and-beyond-burgers-5190889
https://www.simplyrecipes.com/how-to-cook-a-burger-indoors-on-the-stove-5190856
https://www.simplyrecipes.com/tom-collins-recipe-5190817
https://www.simplyrecipes.com/strawberry-buttercream-frosting-recipe-5190772
https://www.simplyrecipes.com/onion-jam-recipe-5190769
https://www.simplyrecipes.com/dijon-mustard-recipe-5190761
https://www.simplyrecipes.com/honey-mustard-recipe-5190758
https://www.simplyrecipes.com/hamburger-buns-recipes-5190754
https://www.simplyrecipes.com/spicy-buffalo-chicken-burgers-recipes-5190753
https://www.simplyrecipes.com/salmon-burgers-wi

In [16]:
table = table.fillna(value = np.nan)
table = table.dropna()
table

,titles,category,Prep time,servings,Ingreds
0,French Buttercream,Most Recent,21 mins,4 cups of frosting,"[1 1/4 cups (300g) granulated sugar, 1/4 cup w..."
1,Tom Collins,Most Recent,5 mins,1 serving,"[2 ounces Old Tom Gin, 3/4 ounce lemon juice, ..."
2,Strawberry Buttercream Frosting,Most Recent,20 mins,40 servings,"[1 1/2 pounds (680g) unsalted butter, at room ..."
3,Onion Jam,Most Recent,45 mins,16 servings,"[4 medium (1 1/2 pounds) yellow onions, medium..."
4,Dijon Mustard,Most Recent,15 mins,14 servings,[8 tablespoons (80 grams) whole brown mustard ...
...,...,...,...,...,...
1497,Persimmon Pomegranate Fruit Salad,Fruit Salads,15 mins,4 servings,"[3 fuyu persimmons, peeled, chopped (1/4 to 1/..."
1498,Berries and Banana Fruit Salad,Fruit Salads,15 mins,6to 8 servings,"[2 cups hulled, roughly chopped strawberries, ..."
1499,Mexican Fruit Cocktail,Fruit Salads,15 mins,3to 4 servings,"[2 cups of cubed pineapple, slice the cubes in..."
1500,Strawberry Nectarine Fruit Salad,Fruit Salads,10 mins,4 servings,[2 to 3 large white nectarines (3/4 pound to a...
